In [1]:
from __future__ import (print_function, absolute_import)
import os
import numpy as np
import pandas as pd
import keras
import keras.backend as K
from keras import datasets
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

from models import CNN, VGG8
from wide_resnet import WideResidualNetwork
from pulearn.utils import synthesize_pu_labels

from keras.callbacks import (
    ReduceLROnPlateau,
    LearningRateScheduler,
    CSVLogger,
    EarlyStopping,
    ModelCheckpoint)

from keras_tqdm import TQDMNotebookCallback

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"]="1"

global _SESSION
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
_SESSION = tf.Session(config=config)
K.set_session(_SESSION)

Using TensorFlow backend.


# CIFAR110

In [2]:
num_classes = 11
batch_size = 128
epochs = 200
data_augmentation = True
checkpoint = None
# checkpoint = 'model_checkpoint_cifar10_wide_resnet.h5'
title = 'cifar110_vgg8'
# title = 'cifar110_wide_resnet'

In [3]:
(x_train_10, y_train_10), (x_test_10, y_test_10) = datasets.cifar10.load_data()
(x_train_100, y_train_100), (x_test_100, y_test_100) = datasets.cifar100.load_data()

y_train_10 = y_train_10 + 1
y_test_10 = y_test_10 + 1
y_train_100[...] = 0
y_test_100[...] = 0

x_train = np.concatenate((x_train_10, x_train_100), axis=0).astype('float32')
y_train = np.concatenate((y_train_10, y_train_100), axis=0).astype('int8')
x_test = np.concatenate((x_test_10, x_test_100), axis=0).astype('float32')
y_test = np.concatenate((y_test_10, y_test_100), axis=0).astype('int8')

x_train = x_train.astype(K.floatx())
x_test = x_test.astype(K.floatx())

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

def normalize(x):
    """Substract mean and Divide by std."""
    x -= np.array([125.3, 123.0, 113.9], dtype=K.floatx())
    x /= np.array([63.0, 62.1, 66.7], dtype=K.floatx())
    return x


x_train = normalize(x_train)
x_test = normalize(x_test)

In [4]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(100000, 32, 32, 3) (100000, 11)
(20000, 32, 32, 3) (20000, 11)


In [5]:
pct_missing = 0.5
y_train_pu = synthesize_pu_labels(y_train, random_state=40, verbose=True)

y_train_enc = y_train_pu[pct_missing]
from sklearn.utils import class_weight
# y_train_dec = np.argmax(y_train_enc, axis=1)
# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(y_train_dec),
#                                                  y_train_dec)
# class_weight = {}
# for k, w in enumerate(class_weights):
#     class_weight[k] = w

y_train_dec = 1 - y_train_enc[:, 0]
class_weights = class_weight.compute_class_weight('balanced',
                                                 [0, 1],
                                                 y_train_dec)
# class_weights = [1., 2.]
class_weight = {}
class_weight[0] = class_weights[0]
for k in range(1, num_classes):
    class_weight[k] = class_weights[1]

# pu_ratio = np.sum(y_train_enc[:, 1:]) / np.sum(y_train_enc[:, 0])

# class_weight = {0: 1}
# for k in range(1, num_classes):
#     class_weight[k] = 1 / pu_ratio

print('class_weight', class_weight)

('Positive (pct_missing=0.0):', 50000, ' -> ', 50000)
('Positive (pct_missing=0.1):', 50000, ' -> ', 45005)
('Positive (pct_missing=0.2):', 50000, ' -> ', 39933)
('Positive (pct_missing=0.3):', 50000, ' -> ', 34992)
('Positive (pct_missing=0.4):', 50000, ' -> ', 30182)
('Positive (pct_missing=0.5):', 50000, ' -> ', 24970)
('Positive (pct_missing=0.6):', 50000, ' -> ', 19969)
('Positive (pct_missing=0.7):', 50000, ' -> ', 14799)
('Positive (pct_missing=0.8):', 50000, ' -> ', 10048)
('Positive (pct_missing=0.9):', 50000, ' -> ', 4956)
('Positive (pct_missing=1.0):', 50000, ' -> ', 0)
class_weight {0: 0.66640010662401705, 1: 2.0024028834601522, 2: 2.0024028834601522, 3: 2.0024028834601522, 4: 2.0024028834601522, 5: 2.0024028834601522, 6: 2.0024028834601522, 7: 2.0024028834601522, 8: 2.0024028834601522, 9: 2.0024028834601522, 10: 2.0024028834601522}


In [6]:
# Checkpoint
checkpointer = ModelCheckpoint(
    monitor='val_acc',
    filepath="model_checkpoint_{}_eu_{}.h5".format(title, pct_missing),
    verbose=1,
    save_best_only=True)

# csvlogger
csv_logger = CSVLogger(
    'csv_logger_{}_eu_{}.csv'.format(title, pct_missing))

# EarlyStopping
early_stopper = EarlyStopping(monitor='val_acc',
                              min_delta=0.001,
                              patience=50)
# Reduce lr with schedule
# def schedule(epoch):
#     lr = K.get_value(self.model.optimizer.lr)
#     if epoch in [60, 120, 160]:
#         lr = lr * np.sqrt(0.1)
#     return lr
# lr_scheduler = LearningRateScheduler(schedule)

# Reduce lr on plateau
lr_reducer = ReduceLROnPlateau(monitor='val_acc',
                               factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=20,
                               min_lr=0.3e-6)

# Loss function switch
alpha = K.variable(0.)
class LossSwitch(keras.callbacks.Callback):
    def __init__(self, alpha):
        self.alpha = alpha       
    def on_epoch_end(self, epoch, logs={}):
        if epoch < 0:
            K.set_value(self.alpha, .05)
#             K.set_value(self.alpha, K.get_value(self.alpha) * (0.8 ** epoch))
        else:
            K.set_value(self.alpha, 0.)
loss_switch = LossSwitch(alpha)

In [7]:
if title == 'cifar110_vgg8':
    model = VGG8(input_shape=x_train.shape[1:], num_classes=num_classes)
elif title == 'cifar110_wide_resnet':
    model = WideResidualNetwork(depth=28, width=8, dropout_rate=0.5,
                                classes=num_classes, include_top=True,
                                weights=None)
if checkpoint is not None:
    model = load_model(checkpoint)
    
from pulearn.losses.keras_losses import unlabeled_exponential_loss
def fade_in_unlabeled_exponential_loss(y_true, y_pred):
    return alpha * keras.losses.categorical_crossentropy(y_true, y_pred) \
        + (1 - alpha) * unlabeled_exponential_loss(y_true, y_pred)

optimizer = keras.optimizers.Adam(1e-3)
model.compile(loss=fade_in_unlabeled_exponential_loss,
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
__________

In [8]:
if not data_augmentation:
    print('No data augmentation applied.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              class_weight=class_weight,
              shuffle=True,
              callbacks=[csv_logger, checkpointer, early_stopper]) 
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        # randomly rotate images in the range (degrees, 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train_pu[pct_missing],
                                     batch_size=batch_size, shuffle=True),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        class_weight=class_weight,
                        callbacks=[csv_logger, checkpointer, early_stopper,
                                   lr_reducer, TQDMNotebookCallback(),
                                  loss_switch])
    model.save('{}_eu_{}.h5'.format(title, pct_missing))
    
    # Predict
    y_train_pred_enc = model.predict(x_train)
    y_test_pred_enc = model.predict(x_test)
    
    lst = []
    y_train_pred = np.argmax(y_train_pred_enc, axis=-1)
    y_train_true = np.argmax(y_train, axis=-1)
    y_train_label = np.argmax(y_train_pu[pct_missing], axis=-1)
    for y_pred, y_true, y_label in zip(y_train_pred, y_train_true, y_train_label):
        lst.append(dict(y_pred=y_pred, y_true=y_true, y_label=y_label, split='train'))

    y_test_pred = np.argmax(y_test_pred_enc, axis=-1)
    y_test_true = np.argmax(y_test, axis=-1)
    for y_pred, y_true in zip(y_test_pred, y_test_true):
        lst.append(dict(y_pred=y_pred, y_true=y_true, y_label=y_true, split='test'))
    
    df = pd.DataFrame(lst)
    df.to_csv('prediction_{}_eu_{}.csv'.format(title, pct_missing), index=False, mode='a')

Using real-time data augmentation.


Epoch 1/200
781/781 [==============================] - 65s - loss: 1.2420 - acc: 0.2399 - val_loss: 0.9732 - val_acc: 0.4678


Epoch 2/200
781/781 [==============================] - 90s - loss: 0.9844 - acc: 0.3616 - val_loss: 0.8417 - val_acc: 0.5194


Epoch 3/200
781/781 [==============================] - 89s - loss: 0.8969 - acc: 0.4009 - val_loss: 0.8104 - val_acc: 0.5208


Epoch 4/200
781/781 [==============================] - 89s - loss: 0.8337 - acc: 0.4256 - val_loss: 0.7457 - val_acc: 0.6176


...

Epoch 196/200
781/781 [==============================] - 45s - loss: 0.2441 - acc: 0.7222 - val_loss: 0.7647 - val_acc: 0.8156


Epoch 197/200
781/781 [==============================] - 45s - loss: 0.2427 - acc: 0.7214 - val_loss: 0.7776 - val_acc: 0.8158


Epoch 198/200
781/781 [==============================] - 45s - loss: 0.2394 - acc: 0.7231 - val_loss: 0.8024 - val_acc: 0.8150


Epoch 199/200
781/781 [==============================] - 46s - loss: 0.2450 - acc: 0.7205 - val_loss: 0.7785 - val_acc: 0.8164


Epoch 200/200
781/781 [==============================] - 45s - loss: 0.2432 - acc: 0.7216 - val_loss: 0.7959 - val_acc: 0.8146

